Use our trained StyleGAN2 model to generate a movie of foram latent space.
Taken from the last section of the demo notebook here:

https://github.com/metazool/stylegan-art/blob/master/styleganportraits.ipynb


In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
home = '/content/drive/My Drive/Forams'
os.chdir(home)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys, os
home = '/content/drive/My Drive/Forams'
sys.path.append(os.path.join(home, 'stylegan2'))
os.chdir(home)
import os
import pickle
import numpy as np
import math
import random
from tqdm import tqdm_notebook as tqdm

import dnnlib.tflib as tflib

tflib.init_tf()

Load the model whose latents we want to visualise

In [4]:

model = os.path.join(home, 'results/00026-stylegan2-tfrecords-1gpu-config-f/network-snapshot-000048.pkl')

truncation = 0.7

with open(model, 'rb') as f:
    _G, _D, Gs = pickle.load(f)

fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
#synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=8)

Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.


Functions to sample the model and create series of intermediate images

In [0]:
def generate_images(latents, labels):
    batch_size = 8
    num_frames = latents.shape[0]
    num_batches = int(np.ceil(num_frames/batch_size))
    images = []
    for b in tqdm(range(num_batches)):
        new_images = Gs.run(latents[b*batch_size:min((b+1)*batch_size, num_frames-1), :], None, truncation_psi=truncation, randomize_noise=False, output_transform=fmt)
        for img in new_images:
            images.append(img)
    return images

def make_movie(images, out_dir, out_name):
    temp_dir = 'frames%06d'%int(1000000*random.random())
    os.system('mkdir %s'%temp_dir)
    for idx in tqdm(range(len(images))):
        skimage.io.imsave(f'{temp_dir}/frame{idx:05}.png', images[idx])
    # This doesn't throw errors on Google Colab / Drive, but doesn't generate the file either.
    # cmd = f'ffmpeg -i {temp_dir}/frame%05d.png -c:v libx264 -pix_fmt yuv420p {home}/{out_name}.mp4' 
    # print(cmd)
    # os.system(cmd)
    # os.system('rm -rf %s'%temp_dir)

def random_sample(num_images, scale, truncation=0.5, seed=None):
    seed = seed if seed is not None else 1000*random.random()
    latents = np.random.RandomState(int(seed)).randn(num_images, *Gs.input_shapes[0][1:])
    print(latents.shape)
    labels = np.zeros([latents.shape[0]])
    images = Gs.run(latents, None, truncation_psi=truncation, randomize_noise=False, output_transform=fmt)
    print(images.shape)
    images_ct = np.concatenate([img for img in images], axis=1)
    print(images_ct.shape)
    return images, latents

def get_latent_interpolation(endpoints, num_frames_per, mode, shuffle):
    if shuffle:
        random.shuffle(endpoints)
    num_endpoints, dim = len(endpoints), len(endpoints[0])
    num_frames = num_frames_per * num_endpoints
    endpoints = np.array(endpoints)
    latents = np.zeros((num_frames, dim))
    for e in range(num_endpoints):
        e1, e2 = e, (e+1)%num_endpoints
        for t in range(num_frames_per):
            frame = e * num_frames_per + t
            r = 0.5 - 0.5 * np.cos(np.pi*t/(num_frames_per-1)) if mode == 'ease' else float(t) / num_frames_per
            latents[frame, :] = (1.0-r) * endpoints[e1,:] + r * endpoints[e2,:]
    return latents

In [6]:
images, latents = random_sample(60, scale=4, truncation=truncation)
import skimage.io


latents = get_latent_interpolation(latents, 30, 'linear', False)


(60, 512)
(60, 256, 256, 1)
(256, 15360, 1)


In [7]:
labels = np.zeros([latents.shape[0]] + [256,256, 1])

images = generate_images(latents, labels)    

make_movie(images, '.', 'latent_forams')

As Drive doesn't want to persist our .mp4 we download the resulting output directory ({home}/framesXXXXXX) and do something like this locally:
```
mkdir tmp
mogrify -path tmp -border 128 -bordercolor grey *.png
cd tmp
ffmpeg -i frame%05d.png -c:v libx264 -pix_fmt yuv420p latent_forams.mp4
ffmpeg -t 2.5 -i latent_forams.mp4 -f gif latent_forams.gif
```

